In [1]:
"""
https://www.cryptocompare.com
jan.allema@gmail.com
PW: jancrypto


https://www.cryptocompare.com/coins/guides/how-to-use-our-api/
https://min-api.cryptocompare.com/documentation
"""

'\nhttps://www.cryptocompare.com\njan.allema@gmail.com\nPW: jancrypto\n\n\nhttps://www.cryptocompare.com/coins/guides/how-to-use-our-api/\nhttps://min-api.cryptocompare.com/documentation\n'

In [2]:
"""
"
API
jancrypto

price poll current and historical
"
"""""

'\n"\nAPI\njancrypto\n\nprice poll current and historical\n"\n'

In [3]:
import cryptocompare
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
import openpyxl

In [4]:
mycrypto = ['BTC', 'ETH', 'MIOTA', 'ADA', 'XLM', 'LINK']

In [5]:
cryptocompare.cryptocompare._set_api_key_parameter('7868ca59e38dde44d5b44e64a0a1a576c059693dfbfe2a7a08a5a2dc5da31957')

'&api_key=7868ca59e38dde44d5b44e64a0a1a576c059693dfbfe2a7a08a5a2dc5da31957'

In [6]:
import os

In [7]:
df_hist_all = pd.DataFrame()

for fname in os.listdir('./data/'):
    df = pd.read_csv('./data/' + fname, index_col=0)
    df_hist_all = pd.concat([df_hist_all,df]).reset_index(drop = True)
df_hist_all['date'] = pd.to_datetime(df_hist_all.date)

df_hist_all = df_hist_all.sort_values(by = 'date').drop_duplicates(subset = 'date').reset_index(drop = True)
df_hist_all

,date,BTC,ETH,MIOTA,ADA,XLM,LINK
0,2019-01-01,3747.39,133.14,0.3606,0.04082,0.1097,0.2906
1,2019-01-02,3880.15,141.90,0.3753,0.04245,0.1157,0.3043
2,2019-01-03,3961.01,156.10,0.4013,0.04539,0.1195,0.3192
3,2019-01-04,3835.86,149.10,0.3866,0.04234,0.1132,0.3706
4,2019-01-05,3874.06,156.04,0.3829,0.04389,0.1152,0.4449
...,...,...,...,...,...,...,...
1034,2021-10-31,61891.91,4323.94,1.3920,1.95500,0.3587,29.7900
1035,2021-11-01,61349.75,4290.10,1.4660,1.96600,0.3732,30.0200
1036,2021-11-02,60960.98,4322.79,1.4040,1.94900,0.3663,31.6900
1037,2021-11-03,63260.05,4593.15,1.3930,1.97100,0.3824,32.2800


In [23]:
try:
    myportfolio = pd.read_excel('C:\\Users\\janallemann\\OneDrive\\Dokumente\\crypto_portfolio.xlsx', skiprows = 16, nrows = 7, header = 0)\
                .iloc[:,[0,10]]\
                .rename(columns = {'Unnamed: 0': 'currency',
                                  'Unnamed: 10': 'amount'})
except:
    myportfolio = pd.read_excel('C:\\Users\\janal\\OneDrive\\Dokumente\\crypto_portfolio.xlsx', skiprows = 16, nrows = 7, header = 0)\
                .iloc[:,[0,10]]\
                .rename(columns = {'Unnamed: 0': 'currency',
                                  'Unnamed: 10': 'amount'})

myportfolio['current_price'] = 0.0
myportfolio['previous_price'] = 0.0
myportfolio.set_index('currency', inplace = True)
days_previous = 7
myportfolio

,amount,current_price,previous_price
currency,,,
USD,NaN,0.0,0.0
BTC,NaN,0.0,0.0
ETH,NaN,0.0,0.0
MIOTA,NaN,0.0,0.0
XLM,NaN,0.0,0.0
LINK,NaN,0.0,0.0
ADA,NaN,0.0,0.0


In [22]:
for coin in mycrypto:
    
    prev_day = datetime.datetime.now() - datetime.timedelta(days=days_previous)
    
    price = cryptocompare.get_price(coin, 'USD')[coin]['USD']
    myportfolio.current_price[coin] = price
    
    prev_price = float(df_hist_all[df_hist_all.date == str(prev_day.date())][coin])
    myportfolio.previous_price[coin] = prev_price

myportfolio['current_value'] = myportfolio.amount*myportfolio.current_price
tot_current = myportfolio.current_value.sum()

myportfolio['previous_value'] = myportfolio.amount*myportfolio.previous_price
tot_previous = myportfolio.previous_value.sum()

myportfolio['diff'] = myportfolio.current_value - myportfolio.previous_value

print('portfolio worth is {:.2f} USD ({:.2f}% in {} day(s))'.format(tot_current,
                                            (tot_current-tot_previous)/tot_current*100,
                                                                   days_previous))

print('equivalent of {:.2f} BTC'.format(tot_current/myportfolio.current_price['BTC']))

print('profit: {:.2f} USD'.format(myportfolio.current_value.sum()-29769))

try:
    wbkName = 'C:\\Users\\janallemann\\OneDrive\\Dokumente\\crypto_portfolio.xlsx'
    wbk = openpyxl.load_workbook(wbkName)
    wks = wbk.worksheets[0]
    wks.cell(row=3, column=2).value = myportfolio.current_price['BTC']
    wks.cell(row=4, column=2).value = myportfolio.current_price['ETH']
    wks.cell(row=5, column=2).value = myportfolio.current_price['MIOTA']
    wks.cell(row=7, column=2).value = myportfolio.current_price['ADA']
    wks.cell(row=8, column=2).value = myportfolio.current_price['XLM']
    wks.cell(row=9, column=2).value = myportfolio.current_price['LINK']
    wbk.save(wbkName)
    wbk.close()

except:
    wbkName = 'C:\\Users\\janal\\OneDrive\\Dokumente\\crypto_portfolio.xlsx'
    wbk = openpyxl.load_workbook(wbkName)
    wks = wbk.worksheets[0]
    wks.cell(row=3, column=2).value = myportfolio.current_price['BTC']
    wks.cell(row=4, column=2).value = myportfolio.current_price['ETH']
    wks.cell(row=5, column=2).value = myportfolio.current_price['MIOTA']
    wks.cell(row=7, column=2).value = myportfolio.current_price['ADA']
    wks.cell(row=8, column=2).value = myportfolio.current_price['XLM']
    wks.cell(row=9, column=2).value = myportfolio.current_price['LINK']
    wbk.save(wbkName)
    wbk.close()

myportfolio

portfolio worth is 55528.18 USD (4.93% in 7 day(s))
equivalent of 0.84 BTC
profit: 25759.18 USD


,amount,current_price,previous_price,current_value,previous_value,diff
currency,,,,,,
USD,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000
BTC,0.167990,66097.9000,61349.7500,11103.806050,10306.162907,797.643143
ETH,5.589683,4726.2500,4290.1000,26418.240177,23980.299853,2437.940323
MIOTA,7758.000000,1.3670,1.4660,10605.186000,11373.228000,-768.042000
XLM,2014.800000,0.3751,0.3732,755.751480,751.923360,3.828120
LINK,18.380000,33.4600,30.0200,614.994800,551.767600,63.227200
ADA,2964.700000,2.0340,1.9660,6030.199800,5828.600200,201.599600


In [ ]:
for col in df_hist_all.columns[1:]:
    plt.figure(figsize = (20,10))
    plt.plot(df_hist_all['date'], df_hist_all[col])
    plt.title(col)

In [ ]:
plt.figure(figsize = (20,10))
for col in df_hist_all.columns[1:]:
    plt.plot(df_hist_all['date'],df_hist_all[col]/df_hist_all[col].max()*df_hist_all['BTC'].max(), label = col)

plt.xlim(datetime.datetime(2021,1,2),
        datetime.datetime(2021,10,26))
plt.legend()
plt.title('max normalized to BTC')
plt.show()

In [ ]:
for col in df_hist_all.columns[2:]:
    plt.figure(figsize = (20,10))
    plt.plot(df_hist_all['date'], df_hist_all['BTC'])
    plt.plot(df_hist_all['date'], df_hist_all[col]/df_hist_all[col].max()*df_hist_all['BTC'].max())
    plt.xlim(datetime.datetime(2021,1,2),
        datetime.datetime(2021,10,26))
    plt.title('BTC and ' + col + ' normalized')

In [ ]:
for col in df_hist_all.columns[2:]:
    plt.figure(figsize = (20,10))
    plt.plot(df_hist_all['date'], df_hist_all[col]/df_hist_all['BTC'])
    plt.xlim(datetime.datetime(2021,1,2),
        datetime.datetime(2021,10,26))
    plt.title(col + ' vs. BTC')